<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#QIIME-Processing" data-toc-modified-id="QIIME-Processing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>QIIME Processing</a></span><ul class="toc-item"><li><span><a href="#Try-colapsing-ASVs-into-99%-OTUs" data-toc-modified-id="Try-colapsing-ASVs-into-99%-OTUs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Try colapsing ASVs into 99% OTUs</a></span><ul class="toc-item"><li><span><a href="#Create-a-new-taxonomy" data-toc-modified-id="Create-a-new-taxonomy-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Create a new taxonomy</a></span></li><li><span><a href="#Remove-bumblebee-sequences-from-table" data-toc-modified-id="Remove-bumblebee-sequences-from-table-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Remove bumblebee sequences from table</a></span></li><li><span><a href="#Save-files" data-toc-modified-id="Save-files-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Save files</a></span></li><li><span><a href="#Create-a-tree" data-toc-modified-id="Create-a-tree-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Create a tree</a></span></li></ul></li><li><span><a href="#Create-a-rarefied-table" data-toc-modified-id="Create-a-rarefied-table-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create a rarefied table</a></span></li><li><span><a href="#Create-a-level-6-taxonomy-table" data-toc-modified-id="Create-a-level-6-taxonomy-table-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create a level-6 taxonomy table</a></span></li></ul></li></ul></div>

# Import Libraries

**Third notebook in analysis** 

**Creates OTU feature table**

In [2]:
import os
from qiime2.plugins import feature_table, taxa, composition, alignment, phylogeny, diversity
from qiime2.plugins.vsearch.methods import cluster_features_de_novo
from qiime2.plugins.feature_classifier.methods import classify_sklearn
from qiime2.plugins.taxa.methods import filter_table
from qiime2 import Artifact, Metadata, Visualization
import pandas as pd
import numpy as np
from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.taxa.methods import collapse
datapath = '../Data'

# QIIME Processing

The feature table, representative sequences, and taxonomy were all calculated separately on the Colby HPC cluster. 

1. Read the feature table, metadata, rep seqs and taxonomy

2. Filter feature table to exclude things not in the metadata

3. Use mafft to align the representative sequences 

4. Use fasttree to make an unrooted tree from the alignment, then midpoint root it 

In [3]:
merged_table = Artifact.load(os.path.join(datapath, 'qiime/merged_table.qza'))
# fix sample ids
mt = merged_table.view(pd.DataFrame)
# make qiime compliant
mt.index = pd.Series(mt.index).apply(lambda x: str(x).replace("-","_"))
# Fix mistakes on 2019 Submission Sheet
mt.index = pd.Series(mt.index).apply(lambda x: 'FJ190628_006' if x == 'Fj190628_006' else x )
mt.index = pd.Series(mt.index).apply(lambda x: 'FJ190827_005' if x == 'FJ190827_005_P3' else x )
mt.index = pd.Series(mt.index).apply(lambda x: 'FJ190827_022' if x == 'FJ190827_005_P4' else x )
# reat back in 
merged_table = Artifact.import_data('FeatureTable[Frequency]', mt)

# get metadata
bombusMeta = Metadata.load(os.path.join(datapath, 'BombusMetadata.tsv'))

# only samples in the metadata 
merged_table = feature_table.methods.filter_samples(table = merged_table, metadata = bombusMeta).filtered_table

rep_seqs = Artifact.load(os.path.join(datapath, 'qiime/merged_rep-seqs.qza'))


## Try colapsing ASVs into 99% OTUs

In [17]:
mt_99, rep_seqs_99 = cluster_features_de_novo(sequences = rep_seqs,
                                table = Artifact.import_data('FeatureTable[Frequency]', mt),
                                perc_identity = .99,threads = 0) 

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --cluster_size /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/tmptewde669 --id 0.99 --centroids /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/q2-DNAFASTAFormat-stpu0cio --uc /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/tmpikss04ir --qmask none --xsize --threads 0 --minseqlength 1 --fasta_width 0



vsearch v2.21.1_macos_x86_64, 8.0GB RAM, 4 cores
https://github.com/torognes/vsearch

Reading file /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/tmptewde669 100%
1349183 nt in 3752 seqs, min 241, max 480, avg 360
Sorting by abundance 100%
Counting k-mers 100%
Clustering 100%
Sorting clusters 100%
Writing clusters 100%
Clusters: 2747 Size min 1, max 27, avg 1.4
Singletons: 2286, 60.9% of seqs, 83.2% of clusters


**filter samples from metadata**

In [18]:
mt_99 = feature_table.methods.filter_samples(table = mt_99, metadata = bombusMeta).filtered_table

### Create a new taxonomy

In [5]:
classifier = Artifact.load(os.path.join(datapath, 'qiime/silva-138-99-nb-classifier.qza'))

In [11]:
taxonomy_99 = classify_sklearn(reads = rep_seqs_99, classifier = classifier,  n_jobs = -2, 
               confidence = 0.7, read_orientation = 'auto')

In [16]:
taxa_99.view(pd.DataFrame)[taxa_99.view(pd.DataFrame)['Taxon'].str.contains('Insec')]

,Taxon,Confidence
Feature ID,,
adb56f9c8af5f07d15e8a475f2ce03ec,d__Eukaryota; p__Arthropoda; c__Insecta; o__Hy...,0.9705258651997525
f6c9d40bde4db43a0620dad7bc17c4d3,d__Eukaryota; p__Arthropoda; c__Insecta; o__Hy...,0.9125041159664535
47b1736403fb15c85e00e56c837c9c8b,d__Eukaryota; p__Arthropoda; c__Insecta; o__Hy...,0.9082495495730497
1c0daf378005442f7b897b1c8b96d48d,d__Eukaryota; p__Arthropoda; c__Insecta; o__Hy...,0.9987674506937081


### Remove bumblebee sequences from table 

investigate

In [22]:
taxa_99 = Artifact.load(os.path.join(datapath, 'qiime', 'taxonomy_99.qza'))



In [23]:
bombus_seqs = pd.DataFrame(rep_seqs_99.view(pd.Series)).join(taxa_99.view(pd.DataFrame\
        )[taxa_99.view(pd.DataFrame)['Taxon'].str.contains('Insec')])
bombus_seqs = bombus_seqs.dropna().copy()
bombus_seqs[0] = bombus_seqs[0].apply(str)

according to blastn, a bee (even after bombus was removed!)

In [38]:
bombus_seqs.loc['adb56f9c8af5f07d15e8a475f2ce03ec', 0]

'GGATTGACAGATTGATAGCTCTTTCTTGATTCGGTGGGTGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAATTCCGATAACGAACGAGACTCTAGCCTGCTAAATAGACGTAACTTATGGTATCTCGAAGGCCCCCGGCTTCGGTCGGTGGGTTTTTACTACCAACGTACAAACAAATCTTCTTAGAGGAACAGGCGGCTTCTAGCCGCACGAGATTGAGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAAGGAATCAGCGTGTTTTCCCTGGCCGAAAGGCCCGGGTAACCCGCTGAACCTCCTTCGTGCTAGGGATTGGGGCTTGCAATTATTCCCCATGAACGAGGAATTCCCAGTAAGCGCGAGTCATAAGCTCGCGTT'

a bee

In [42]:
bombus_seqs.iloc[1][0]

'AGTTGTTGCGGTTAAAAAGCTCGTAGTTGAATCTGTGTGTCACAGTGTCGGTTCACCGCTCGCGGTGTTTAACTGGCATTATGTGGTACGTCCTATCGGTGGGCTTAGCTCCTCGCGGGCGGTCCAACTAATATCCCATCGCGGTGCTCTTCACTGAGTGTCGAGGTGGGCCGATACGTTTACTTTGAACAAATTAGAGTGCTTAAAGCAGGCTACCTTCGCCTGAATACTGTGTGCATGGAATAATGGAATAGGACCTCGGTTCTATTTTGTTGGTTTTCGGAGCCCCGAGGTAATGATTAATAGGGACAGATGGGGGCATTCGTATTGCGACGTTAGAGGTGAAATTCTTGGATCGTCGCAAGACGGACAGAAGCGAAAGCATTTGCCAAAAATGTTTTCATTAATCAAGAAC'

Another bee

In [43]:
bombus_seqs.iloc[2][0]

'GGATTGACAGATTTATAGCTCTTTCTTGATTCGGTGGGTGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAATTCCGATAACGAACGAGACTCTAGCCTGCTAAATAGACGTAACTTATGGTATCACGAAGGTCCCCGACTTCGGTCGGTGGGTTTTTACTACCAACGTACAAACAAATCTTCTTAGAGGGACAGGCGGCTTCTAGCCGCACGAGATTGAGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAAGGAATCAGCGTGTTTTCCCTGGCCGAAAGGCCCGGGTAACCCGCTGAACCTCCTTCGTGCTAGGGATTGGGGCTTGCAATTATTCCCCATGAACGAGGAATTCCCAGTAAGCGCGAGTCATAAGCTCGCGTT'

a bee

In [45]:
bombus_seqs.iloc[3][0]

'GGATTGACAGATTGATAGCTCTTTCTTGATTCGGTGGGTGGTGGTGCATGGCCGTTCTTAGTTGGTGGAGCGATTTGTCTGGTTAATTCCGATAACGAACGAGACTCTAGCCTGTTAAATAGACGTAAATTATGGTATCTCGAAGGCCCCCGACTTCGGTCGGTGTGGTTTTTACTACCAACGTACAAACAAATCTTCTTAGAGGGACAAGCGGCTTCTAGTCGCACGAGATTGAGCAATAACAGGTCTGTGATGCCCTTAGATGTTCTGGGCCGCACGCGCGCTACACTGAAGGAATCAACGTGTTTTCCCTGGCCGAAAGGTCCGGGTAACCCGCTGAAACTCCTTCGTGCTAGGGATTGGGGCTTGAAATTATTCCCCATGAACGAGGAATTCCCAGTAAACGCGAGTCATAAGCTCGCGTT'

besides the bees, there are a bunch of fungus and a couple plant OTUs. They don't add up to a lot though (max 1%)

**save the full table for the manuscript**

In [65]:
taxa_99.view(pd.DataFrame).join(mt_99.view(pd.DataFrame).transpose()).to_csv('../data/table/OTU_table_full.tsv',
                                                                             sep = '\t')

In [67]:
mt_99 = filter_table(table = mt_99, taxonomy = taxa_99,
                    exclude = 'd__Eukaryota; p__Arthropoda; c__Insecta').filtered_table

### Save files 
(they took too long to make to do it again!)
- feature table 
- representative sequences 
- new taxonomy 

In [68]:
mt_99.save(os.path.join(datapath, 'qiime', 'mt_99.qza'))
rep_seqs_99.save(os.path.join(datapath, 'qiime', 'rep_seqs_99.qza'))
taxonomy_99.classification.save(os.path.join(datapath, 'qiime', 'taxonomy_99.qza'))


'../Data/qiime/rep_seqs_99.qza'

### Create a tree

In [5]:
mafft_alignment = alignment.methods.mafft(rep_seqs_99)
masked_mafft_alignment = alignment.methods.mask(mafft_alignment.alignment)
unrooted_tree = phylogeny.methods.fasttree(masked_mafft_alignment.masked_alignment)
rooted_tree_99 = phylogeny.methods.midpoint_root(unrooted_tree.tree)
rooted_tree_99.rooted_tree.save(os.path.join(datapath, 'qiime/rooted_tree_99.qza'))
#rooted_tree_99 = Artifact.load(os.path.join(datapath, 'qiime/rooted_tree_99.qza'))


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/qiime2-archive-m43u23l5/2ebe578c-cd74-48d5-a84d-dea2bd3b814e/data/dna-sequences.fasta



inputfile = orig
2747 x 480 - 241 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 2701 / 2747 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
 2740 / 2747
done.

Progressive alignment 1/2... 
STEP  2501 / 2746 (thread    0) h
Reallocating..done. *alloclen = 1999
STEP  2701 / 2746 (thread    0)
Reallocating..done. *alloclen = 3387

done.

Making a distance matrix from msa.. 
 2700 / 2747 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
 2740 / 2747
done.

Progressive alignment 2/2... 
STEP  2501 / 2746 (thread    0)
Reallocating..done. *alloclen = 1982
STEP  2701 / 2746 (thread    0)
Reallocating..done. *alloclen = 3825

done.

disttbfast (nuc) Version 7.505
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/qiime2-archive-pswn2tf0/505ae797-a596-48ae-818e-49fa5593f2c8/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /var/folders/kk/vx23v0b52wv0sx4vqhls42dw0000gn/T/qiime2-archive-pswn2tf0/505ae797-a596-48ae-818e-49fa5593f2c8/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.48 seconds: Top hits for    640 of   2745 seqs (at seed    100)
      0.61 seconds: Top hits for   1001 of   2745 seqs (at seed    200)
      0.78 seconds: Top hits for   1340 of   2745 seqs (at seed    400)
      0.97 seconds: Top hits for   1701 of   2745 seqs (at seed    600)
      1.09 seconds: Top hits for   1897 of   2745 seqs (at seed    800)
      1.20 seconds: Top hits for   2031 of   2745 seqs (at seed   1000)
      1.37 seconds: Top hits for   2233 of   2745 seqs (at seed   1400)
      1.49 seconds: Top hits for   2316 o

## Create a rarefied table 

In [46]:
mt_99 = Artifact.load(os.path.join(datapath, 'qiime', 'mt_99.qza'))
rooted_tree_99=Artifact.load(os.path.join(datapath, 'qiime/rooted_tree_99.qza'))
bombusMeta = Metadata.load(os.path.join(datapath, 'BombusMetadata.tsv'))

In [47]:
rarefaction = diversity.actions.alpha_rarefaction(table = mt_99,
                                                 max_depth = 15000,
                                                 phylogeny = rooted_tree_99,
                                                 metadata = bombusMeta)
rarefaction.visualization.save(os.path.join(datapath, 'qiime/rarefaction.qzv'))
rarefaction = Visualization.load(os.path.join(datapath, 'qiime/rarefaction.qzv'))

**4k looks good**
- shannon plateaus, total features still increases for a bit, but assuming that theyre fringe otus

In [51]:
#rarefaction

In [80]:
mt_99_rare = feature_table.methods.rarefy(table = mt_99,
                                          sampling_depth = 4000)

In [83]:
mt_99_rare.rarefied_table.view(pd.DataFrame).shape

(631, 1519)

In [84]:
mt_99.view(pd.DataFrame).shape

(638, 2388)

In [16]:
mt_99_rare.rarefied_table.save(os.path.join(datapath, 'qiime', 'mt_99_rare.qza'))

'../Data/qiime/mt_99_rare.qza'

## Create a level-6 taxonomy table

In [75]:
mt_99 = Artifact.load(os.path.join(datapath, 'qiime', 'mt_99.qza'))
taxa_99 = Artifact.load(os.path.join(datapath, 'qiime', 'taxonomy_99.qza'))


In [77]:
mt_99_c6 = collapse(mt_99, taxa_99, 6).collapsed_table
mt_99_c6.save(os.path.join(datapath, 'qiime', 'mt_99_c6.qza'))


'../Data/qiime/mt_99_c6.qza'